This notebook is part of the $\omega radlib$ documentation: https://docs.wradlib.org.

Copyright (c) $\omega radlib$ developers.
Distributed under the MIT License. See LICENSE.txt for more info.

# xarray powered Cf/Radial and ODIM_H5

In this example, we read and write Cf/Radial (NetCDF) and ODIM_H5 (HDF5) data files from different sources using an xarray powered data structure.

In [ ]:
import wradlib as wrl
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as pl
import numpy as np
import xarray as xr
try:
    get_ipython().magic("matplotlib inline")
except:
    pl.ion()

In [ ]:
from wradlib.io.xarray import CfRadial, OdimH5

## Load ODIM_H5 Volume Data

In [ ]:
fpath = 'hdf5/knmi_polar_volume.h5'
f = wrl.util.get_wradlib_data_file(fpath)
cf1 = OdimH5(f)

# Inspect root group

You can use the object dictionary using cf1['root'] or the object attribute cf1.root.

The `sweep` dimension contains the number of scans in this radar volume. Further the dataset consists of variables (location coordinates, time_coverage) and attributes (Conventions, metadata).

In [ ]:
cf1.root

### Inspect sweep group(s)

Same as with the root-group there are the two options to get access. The dimensions consist of `range` and `time` with added coordinates `azimuth`, `elevation`, `range` and `time`. There will be variables like radar moments (DBZH etc.) and sweep-dependend metadata (like `fixed_angle`, `sweep_mode` etc.).

In [ ]:
cf1['sweep_1']

### Inspect radar moments

The dataarrays can be accessed by key or by attribute. Each dataarray has the datasets dimensions and coordinates of it's parent dataset. There are attributes connected which are defined by Cf/Radial and/or ODIM_H5 standard.

In [ ]:
cf1.sweep_1.DBZH

### Create simple plot

Using xarray features a simple plot can be created like this. Note the `sortby('time')` method, which sorts the radials by time.

In [ ]:
cf1.sweep_1.sortby('time').DBZH.plot()

### Export data to ODIM_H5 and Cf/Radial2

In [ ]:
cf1.to_odim('testodim.h5')
cf1.to_cfradial2('test_odim_as_cfradial.nc')

### Import again

In [ ]:
cf1a = OdimH5('testodim.h5')
cf1b = CfRadial('test_odim_as_cfradial.nc')

### Check equality

In [ ]:
xr.testing.assert_equal(cf1.root, cf1a.root)
xr.testing.assert_equal(cf1.sweep_1, cf1a.sweep_1)
xr.testing.assert_equal(cf1.root, cf1b.root)
xr.testing.assert_equal(cf1.sweep_1, cf1b.sweep_1)

### Mask some values

In [ ]:
cf1.sweep_1['DBZH'] = cf1.sweep_1.DBZH.where(cf1.sweep_1.DBZH >= 0)
cf1.sweep_1.DBZH.sortby('time').plot()

## Load Cf/Radial1 Volume Data

In [ ]:
fpath = 'netcdf/cfrad.20080604_002217_000_SPOL_v36_SUR.nc'
f = wrl.util.get_wradlib_data_file(fpath)
cf2 = CfRadial(f)

### Inspect root group

In [ ]:
cf2.root

### Inspect sweep group(s)

In [ ]:
cf2.sweep_1

### Inspect radar moments

In [ ]:
cf2.sweep_1.DBZ

### Create simple plot

In [ ]:
cf2.sweep_1.DBZ.plot()

### Export data to Cf/Radial2 and ODIM_H5

In [ ]:
cf2.to_cfradial2('testcfradial2.nc')
cf2.to_odim('test_cfradial_as_odim.h5')

### Import again

In [ ]:
cf2a = CfRadial('testcfradial2.nc')
cf2b = OdimH5('test_cfradial_as_odim.h5')

### Check equality

For Cf/Radial there are issues with nan, which need to be fixed. For the ODIM_H5 intercomparison there are too problems with nan and issues with attributes.

In [ ]:
xr.testing.assert_equal(cf2.root, cf2a.root)
xr.testing.assert_equal(cf2.sweep_1.drop(['DBZ', 'VR']), 
                        cf2a.sweep_1.drop(['DBZ', 'VR']))

xr.testing.assert_allclose(cf2.root.time_coverage_start, 
                           cf2b.root.time_coverage_start)
xr.testing.assert_allclose(cf2.sweep_1.range, cf2b.sweep_1.range)